In [17]:
import numpy as np
import pandas as pd

In [18]:
path = "/scratch/ssd004/scratch/snajafi/checkpoints/natural-instructions-2.8/tasks"
file = "selected_instances_chunk_3.gemma_output.csv"
df = pd.read_csv(f"{path}/{file}", sep=",")
gemma_outputs = df.gemma_output.tolist()
gemma_logits = df.gemma_logit.tolist()

In [19]:
file = "selected_instances_chunk_3.csv"
clean_file = "selected_instances_chunk_3.gemma_output.cleaned.csv"
original_df = pd.read_csv(f"{path}/{file}", sep=",")
instructions = original_df.instruction.tolist()
inputs = original_df.input.tolist()
outputs = original_df.output.tolist()
ids = original_df.id.tolist()

In [20]:
unique_instructions = list(set(instructions))
scores_per_instruction = {instruction: [] for instruction in unique_instructions}
indices_per_instruction = {instruction: [] for instruction in unique_instructions}

for idx, logit in enumerate(gemma_logits):
    scores_per_instruction[instructions[idx]].append(logit)
    indices_per_instruction[instructions[idx]].append(idx)

print(len(scores_per_instruction.keys()))

1545


In [21]:
zscores_per_instruction = {instruction: None for instruction in unique_instructions}
for instruction, logits in scores_per_instruction.items():
    if logits:
        logits_arr = np.array(logits)
        mean_arr = np.mean(logits_arr)
        std_arr = np.std(logits_arr)
        zscores = (logits_arr - mean_arr) / (std_arr + 1e-16)
        zscores_per_instruction[instruction] = zscores

keep_per_instruction = {instruction: [] for instruction in unique_instructions}
for instruction, zscores in zscores_per_instruction.items():
    if zscores is not None:
        for idx, score in enumerate(zscores):
            if score >= 0.0 and score <= 2.0:
                keep_per_instruction[instruction].append(indices_per_instruction[instruction][idx])

In [22]:
keep_rows = []
for instruction, indices in keep_per_instruction.items():
    if indices:
        for idx in indices:
            row = {
                "instruction": instruction,
                "id": ids[idx],
                "input": inputs[idx],
                "output": outputs[idx],
                "gemma_logit": gemma_logits[idx],
                "gemma_output": gemma_outputs[idx],
            }
            keep_rows.append(row)

to_write_df = pd.DataFrame.from_dict(keep_rows)
to_write_df.to_csv(f"{path}/{clean_file}", header=True, sep=",", index=False)

In [23]:
print(len(keep_rows))

1938


In [24]:
print(len(ids))

40221
